# Azure Machine Learning Service

Azure Machine Learning (AML) is Microsoft’s cloud service designed to support the entire data science lifecycle, enabling model development, training, and deployment. It provides a collaborative environment with MLOps capabilities to monitor, retrain, evaluate, and redeploy models. This chapter focuses on deploying an AML workspace, which integrates Azure resources to create an environment for managing AML assets. 

Deployment can be done in three ways: (1) **Guided User Interface** (GUI) within the [Azure Portal](https://azure.microsoft.com/en-us/get-started/azure-portal), (2) **Azure CLI**  [`ml` extension (v2)](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cli?view=azureml-api-2&tabs=public) for operations via the terminal, and (3) **Azure Resource Management** (ARM) templates. Key resources deployed include:

| Resource                  | Purpose                                   |
|---------------------------|-------------------------------------------|
| AML Studio            | A portal for managing workloads.          |
| Azure Key Vault       | For storing sensitive information.        |
| Application Insights  | For logging and monitoring.               |
| Azure Container Registry | For storing Docker images.             |
| Azure Storage Account | For holding data.                         |

These resources support activities like creating compute resources and coding in preferred IDEs such as Jupyter Notebook, Jupyter Lab, and VS Code. AML streamlines the entire ML workflow, from development to deployment, in a scalable and efficient manner.

<br>

## Creating an AMLS workspace

Every workspace must be assigned to an Azure subscription, which is where billing happens. You use resource groups like folders to organize and manage resources, including the workspace you're about to create.

```{figure} ../../img/azure/01-create-workspace.png
---
name: 01-create-workspace
width: 90%
align: center
---
Creating a workspace via the GUI. Clicking on the **Review + create** button validates the filled in values, before resources are deployed.
```

Every workspace is assigned under an **Azure subscription**, which is a single billing unit for services under the same logical grouping as required by business or technical considerations. Next, **resource groups** are like folders used to organize and manage resources. For example, different environments such as *dev*, *staging*, and *prod* can have its own resource group. As we deploy the AMLS workspace, the resources will be deployed into this resource group. This allows us, for example, to easily delete the resources after performing this exercise.

```{figure} ../../img/azure/01-resource-visualizer.png
---
name: 01-resource-visualizer
width: 100%
align: center
---
Visualizing services under the resource group.
```

Clicking **Launch studio** in the workspace leads you to the Azure Machine Learning service. You may have to update Public network access to add your client IP address if you modified network access during creation. Once inside, you can scroll down to create a **compute instance** ({numref}`01-compute-instance`) which is simply an [Azure virtual machine](https://learn.microsoft.com/en-us/azure/virtual-machines/overview) (VM). Note that VMs of [different sizes](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/overview)[^1] are available.

[^1]: The choice of VM depends on the use-case. For example, whether to use a high number of cores for multiprocessing. RAM of a VM is also important: a good rule of
thumb is to have 20 times the amount of RAM as the size of your data in CSV format. And similarly for other parameters. In general, VMs are classified as either, general purpose, or compute or memory optimized.

<br>

```{figure} ../../img/azure/01-compute-instance.png
---
name: 01-compute-instance
width: 100%
align: center
---
Launching VS Code in a compute instance.
```

```{note}
A compute instance in AML has a dedicated **assigned user** who has root access. This single-user assignment ensures that all actions performed on the compute instance use that user's identity for Azure RBAC (Role-Based Access Control) and attribution of experiment jobs.
```

### CLI

The [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/) allows a user to connect to Azure and execute administrative commands on Azure resources via the terminal. It allows the execution of terminal commands using interactive prompts or a script. CLI commands follow the format:

```
az ml <noun> <verb> <options>
```

Note CLI version must be ≥2.15.0 to use the `ml` extension installed as follows:

| Command                                      | Description                                                                 |
|----------------------------------------------|-----------------------------------------------------------------------------|
| `az version` | Show version of the installed Azure CLI. |
| `az extension remove -n ml`                  | Remove any existing installation of the `ml` extension.                     |
| `az extension add -n ml`                     | Install the `ml` extension.                                                 |
| `az ml -h`                                   | Help command to verify installation and list subcommands.  |
| `az extension update -n ml`                  | Upgrade the `ml` extension to the latest version.                           |

The following command opens a window to automatically log into the Azure CLI:

```zsh
az login
```

Then, we can create the workspace and its resource group as follows:

```zsh
export LOC="eastus2"
export GROUP="aml-resource-group-cli-test"
export WORKSPACE="aml-workspace-cli-test"

az group create --name $GROUP --location $LOC
az ml workspace create -n $WORKSPACE -g $GROUP -l $LOC
```

To create a compute instance:

```bash
az ml compute create --name compute-cli \
    --size STANDARD_D3_V2 --type ComputeInstance \
    --resource-group $GROUP --workspace-name $WORKSPACE
```

The resulting workspace has public network access enabled by default, which can later be changed via the GUI once created. The next approach offers further customization using [declarative syntax](https://en.wikipedia.org/wiki/Declarative_programming).

### ARM templates

ARM templates provides a convenient and repeatable method to automate and parameterize the creation of Azure resources. Here we write a simple ARM template to build an AMLS workspace deployed using the Azure CLI. Note that [Quickstart Templates](https://github.com/Azure/azure-quickstart-templates/tree/master/quickstarts/microsoft.machinelearningservices/machine-learning-workspace) are available in GitHub.

In [2]:
%%bash
curl -O https://raw.githubusercontent.com/Azure/azure-quickstart-templates/refs/heads/master/quickstarts/microsoft.machinelearningservices/machine-learning-workspace/azuredeploy.json &&
cat azuredeploy.json | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5015  100  5015    0     0   112k      0 --:--:-- --:--:-- --:--:--  113k


{
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
  "contentVersion": "1.0.0.0",
  "metadata": {
    "_generator": {
      "name": "bicep",
      "version": "0.13.1.58284",
      "templateHash": "17277793070204216233"
    }
  },
  "parameters": {
    "name": {
      "type": "string",
      "metadata": {
        "description": "Specifies the name of the deployment."
      }
    },
    "environment": {
      "type": "string",
      "metadata": {
        "description": "Specifies the name of the environment."
      }
    },
    "location": {
      "type": "string",
      "allowedValues": [
        "australiaeast",
        "brazilsouth",
        "canadacentral",
        "centralus",
        "eastasia",
        "eastus",
        "eastus2",
        "francecentral",
        "japaneast",
        "koreacentral",
        "northcentralus",
        "northeurope",
        "southeastasia",
        "southcentralus",
        "uksouth",
        "westcentr

This configuration describes the minimal set of resources you require to create an AML workspace. It has two main parts, required `parameters` and  `resources`. It creates the following Azure services that is associated to the AML workspace:

| Provider and type | Service instance |
| - | - |
| `Microsoft.Insights/components` | Azure Application Insights |
| `Microsoft.KeyVault/vaults` | Azure Key Vault |
| `Microsoft.Storage/storageAccounts` | Azure Storage |
| `Microsoft.ContainerRegistry/registries` | Azure Container Registry |
| `Microsoft.MachineLearningServices/workspaces` | Azure Machine Learning workspace |


The sample template has three parameters: `name`, `environment`, and `location`. We now create the workspace using Azure CLI. First, create the **resource group**, and then **deploy** the ARM template. Note that the name parameter has to be *globally* unique. The following creates the entire stack:

```bash
export DEPLOYMENT_NAME="example-deployment"
export GROUP="aml-resource-group-arm-test"
export NAME="un1qu3"
export ENV="test"
export LOC="eastus2"

az group create --name $GROUP --location $LOC
az deployment group create \
    --name $DEPLOYMENT_NAME --resource-group $GROUP \
    --template-file "azuredeploy.json" \
    --parameters name=$NAME environment=$ENV location=$LOC
```

Workspace configuration can be manually changed in the GUI, then the updated ARM template can be exported under **Automation>Export Template**. For example, enable public network access from selected IP addresses in **Settings>Networking**. This adds the following in the resulting ARM which can then be added in `workspace-deploy.json`:

```json
"properties": {
    "publicNetworkAccess": "Enabled",
    "ipAllowlist": [
        "123.123.8.123/32"
    ],
}
```

The deployments can be found by accessing the resource group **Setting>Deployments**:


```{figure} ../../img/azure/01-deployments.png
---
name: 01-deployments
width: 100%
align: center
---
```
